In [1]:

def clear_memory():
    !sudo sysctl -w vm.drop_caches=3
clear_memory()


Coloque su dedo en el lector de huellas dactilares
vm.drop_caches = 3


In [2]:
import pandas as pd
import polars as pl
import ManipulateData
import MetodoLexico
import DataLoader_Polars
from rapidfuzz import fuzz
#import jellyfish
#import re


In [3]:
data_loader = DataLoader_Polars.DataLoaderPolars("../DataSets_CSVs/DataSets_Parquets/")
df_concesiones = data_loader.load_concesiones()
manipulator = ManipulateData.ManipulateData(df_concesiones, "", "")

In [4]:
'''
#Para buscar cuales pueden ser las metricas y metodos lexicos correctos a utilizar hacemos una extracción de los titulares únicos a los que nos enfrentamos
titulares_unicos = df_concesiones.sort('titular').select('titular').unique()
'''

"\n#Para buscar cuales pueden ser las metricas y metodos lexicos correctos a utilizar hacemos una extracción de los titulares únicos a los que nos enfrentamos\ntitulares_unicos = df_concesiones.sort('titular').select('titular').unique()\n"

---
## Análisis de anomlias en el df_concesiones relaciondas con el cambio de Titulares
---

In [5]:
df_concesiones = manipulator.date_string_to_date(df_concesiones)

In [6]:
# Para truncar fechas que ya tienen tiempo
df_concesiones = df_concesiones.with_columns(
    pl.col('ultimo_registro').dt.date().alias('ultimo_registro')
)

In [7]:
#n_nombres_titulares_antes_estandarizar = len(df_concesiones.select('titular').unique())
#n_nombres_titulares_antes_estandarizar

In [8]:
'''
#Para un segundo enfoque podemos normalizar los textos de los titulares con diferentes parámetros

df_concesiones = ManipulateData.ManipulateData(df_concesiones).estandarizar_titular('titular',
                                                                                    sin_espacios=False,
                                                                                    sin_stopword=True,
                                                                                    sin_terminos_mercantiles=True,
                                                                                    quitar_tokens_1_letra=True)
'''

"\n#Para un segundo enfoque podemos normalizar los textos de los titulares con diferentes parámetros\n\ndf_concesiones = ManipulateData.ManipulateData(df_concesiones).estandarizar_titular('titular',\n                                                                                    sin_espacios=False,\n                                                                                    sin_stopword=True,\n                                                                                    sin_terminos_mercantiles=True,\n                                                                                    quitar_tokens_1_letra=True)\n"

In [9]:
#n_nombres_titulares_despues_estandarizar = len(df_concesiones.select('titular').unique())
#n_nombres_titulares_despues_estandarizar

In [10]:
# Seleccionar columnas en el orden deseado

df_concesiones = df_concesiones.select([
    'titulo', 'titular', 'ultimo_registro',
    *[col for col in df_concesiones.columns if col not in ['titulo', 'titular', 'ultimo_registro']]
])


In [11]:
# Ordenar el DataFrame
df_concesiones = df_concesiones.sort([
    pl.col('titulo').count().over('titulo'),
    'titular', 
    'ultimo_registro'
], descending=[True, False, False])

In [12]:
# Obtener información de títulos y titulares
titulos_info = df_concesiones.group_by('titulo').agg([
    pl.col('titular').n_unique().alias('N_titulares')
]).sort('N_titulares', descending=True)



---
##### Analisis de cambios de  volumenes

In [13]:
volumenes_totales_info = df_concesiones.group_by('titulo').agg([
    pl.col('volumen_de_extraccion_anual_de_aguas_nacionales_que_ampara_el_titulo_en_m3').n_unique().alias('cambios_volumen'),
    pl.col('volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3').n_unique().alias('cambios_volumen_subterraneo'),
    pl.col('volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3').n_unique().alias('cambios_volumen_superficial')
])

---
##### Analisis de la la cantidad de periodos y cantidad de cambios

In [14]:
periodos_info = df_concesiones.group_by('titulo').agg([
    pl.col('ultimo_registro').n_unique().alias('cantidad_periodos'),
    pl.col('ultimo_registro').unique().alias('periodos_presentes'),
    pl.col('ultimo_registro').count().alias('N_cambios_ultimo_registro'),
])

In [15]:

# Recrear df_anomalias con los datos
df_anomalias = pl.DataFrame({
    'titulo': titulos_info['titulo'].to_list(),
    'N_titulares': titulos_info['N_titulares'].to_list(),
    'Score': [None] * len(titulos_info),
    'Diferente_titular': [None] * len(titulos_info),
    'Caracteres_distintos': [None]* len(titulos_info),
    'Cambios_en_20_periodos': periodos_info['N_cambios_ultimo_registro'].to_list(), # Cuantas veces aparece el registro en los 20 periodos
    'Periodos': periodos_info['periodos_presentes'].to_list(), # aqui se ponen los periodos en los que hubieron cambios.
    'Cantidad_periodos': periodos_info['cantidad_periodos'].to_list(), #Cuantos periodos hay en total en dicho registro
    'Anomalia_V_Total': volumenes_totales_info['cambios_volumen'].to_list(),
    'Anomalia_V_Subterraneo': volumenes_totales_info['cambios_volumen_subterraneo'].to_list(),
    'Anomalia_V_Superficial': volumenes_totales_info['cambios_volumen_superficial'].to_list()
})

In [16]:
'''
def length_prefilter(a: str, b: str, base_abs: int = 5, rel_long: float = 0.50) -> bool:
    # Semántica: devuelve True si las longitudes son tan distintas
    # que podemos DESCARTAR el par sin seguir a métodos léxicos.
    L1 = len(a or "")
    L2 = len(b or "")
    Lmax = max(L1, L2)
    if Lmax <= 3:
        return False  # no decidas por longitud cuando son ultra cortos

    # Umbral absoluto dinámico (20% de Lmax, al menos base_abs)
    abs_thr = max(base_abs, int(round(0.20 * Lmax)))

    # Umbral relativo: más exigente en cortos
    rel_thr = 0.40 if Lmax < 10 else rel_long  # p.ej. 0.40 para cortos, 0.50 para largos

    gap_abs = abs(L1 - L2)
    gap_rel = gap_abs / Lmax

    return (gap_abs > abs_thr) and (gap_rel > rel_thr)
'''

'\ndef length_prefilter(a: str, b: str, base_abs: int = 5, rel_long: float = 0.50) -> bool:\n    # Semántica: devuelve True si las longitudes son tan distintas\n    # que podemos DESCARTAR el par sin seguir a métodos léxicos.\n    L1 = len(a or "")\n    L2 = len(b or "")\n    Lmax = max(L1, L2)\n    if Lmax <= 3:\n        return False  # no decidas por longitud cuando son ultra cortos\n\n    # Umbral absoluto dinámico (20% de Lmax, al menos base_abs)\n    abs_thr = max(base_abs, int(round(0.20 * Lmax)))\n\n    # Umbral relativo: más exigente en cortos\n    rel_thr = 0.40 if Lmax < 10 else rel_long  # p.ej. 0.40 para cortos, 0.50 para largos\n\n    gap_abs = abs(L1 - L2)\n    gap_rel = gap_abs / Lmax\n\n    return (gap_abs > abs_thr) and (gap_rel > rel_thr)\n'

In [17]:
'''
def score_y_clase(a:str, b:str):
    # Pre-filtrado por longitud
    if length_prefilter(a, b):
        return 0, 0
    else:
        #metricas
        ts = fuzz.token_set_ratio(a, b)
        jw = jellyfish.jaro_winkler_similarity(a, b) * 100
        score = (ts + jw) / 2
        #reglas    
        if ts >= 96 or (ts >= 93 and jw >= 93):
            clase = 1 #iguales
        elif ts < 90 or (ts < 85 and jw < 88):
            clase = 0 #diferentes
        else:
            clase = 2 
    return score, clase
'''

'\ndef score_y_clase(a:str, b:str):\n    # Pre-filtrado por longitud\n    if length_prefilter(a, b):\n        return 0, 0\n    else:\n        #metricas\n        ts = fuzz.token_set_ratio(a, b)\n        jw = jellyfish.jaro_winkler_similarity(a, b) * 100\n        score = (ts + jw) / 2\n        #reglas    \n        if ts >= 96 or (ts >= 93 and jw >= 93):\n            clase = 1 #iguales\n        elif ts < 90 or (ts < 85 and jw < 88):\n            clase = 0 #diferentes\n        else:\n            clase = 2 \n    return score, clase\n'

In [18]:
def score_y_clase_wrapper(a: str, b: str):
    """Wrapper para usar MetodoLexico en lugar de la función score_y_clase"""
    metodo = MetodoLexico.MetodoLexico(a, b)
    return metodo.score_y_clase()


In [19]:
def get_all_metrics(a: str, b: str):
    """Función que retorna todos los valores de una vez para evitar múltiples instanciaciones"""
    metodo = MetodoLexico.MetodoLexico(a, b)
    score, clase, caracteres_conflictivos = metodo.score_y_clase()
    return (score, clase, caracteres_conflictivos)


In [20]:
# Títulos con exactamente 2 titulares únicos
tit2 = (
    df_concesiones
    .select(['titulo','titular'])
    .unique()
    .group_by('titulo')
    .agg(pl.col('titular').unique().alias('titulares'))
    .with_columns(pl.col('titulares').list.len().alias('N_titulares'))
    .filter(pl.col('N_titulares') == 2)
)

# Extrae pares (t1, t2)
tit2_pairs = tit2.with_columns([
    pl.col('titulares').list.get(0).alias('titular1'),
    pl.col('titulares').list.get(1).alias('titular2')
]).select(['titulo','titular1','titular2'])

In [21]:
tit2_scored = (
    tit2_pairs.with_columns([
        pl.struct(['titular1','titular2']).map_elements(
            lambda s: get_all_metrics(s['titular1'], s['titular2']),
            return_dtype=pl.Object  # <- importante si all_metrics es tuple/list
        ).alias('all_metrics')
    ])
    .with_columns([
        pl.col('all_metrics').map_elements(lambda x: x[0]).alias('Score'),
        pl.col('all_metrics').map_elements(lambda x: x[1]).alias('Diferente_titular'),
        pl.col('all_metrics').map_elements(lambda x: x[2]).alias('Caracteres_distintos'),
    ])
    .drop('all_metrics')
)


/tmp/ipykernel_18284/3898457111.py:8: MapWithoutReturnDtypeWarning: 'return_dtype' of function python_udf must be set

A later expression might fail because the output type is not known. Set return_dtype=pl.self_dtype() if the type is unchanged, or set the proper output data type.
  .with_columns([


In [22]:
df_anomalias = (
    df_anomalias.drop(['Score','Diferente_titular', 'Caracteres_distintos'])  # Eliminar las columnas anteriores
    .join(tit2_scored.select(['titulo','Score','Diferente_titular','Caracteres_distintos']), on='titulo', how='left')  # Incluir Caracteres_distintos
    .select([
        'titulo',
        'N_titulares',
        'Score',
        'Diferente_titular',
        'Caracteres_distintos',
        'Cambios_en_20_periodos',
        'Periodos',
        'Cantidad_periodos',
        'Anomalia_V_Total',
        'Anomalia_V_Subterraneo',
        'Anomalia_V_Superficial'
    ])
)

In [23]:

# fechas = df_concesiones.select('ultimo_registro').unique().sort('ultimo_registro')
# titulos_list = df_concesiones.group_by('titulo').agg(pl.count()).sort('count', descending=True)
# nombres_unicos_en_titulares = df_concesiones.sort('titular').select('titular').unique()
# Distribución de títulos por frecuencia
#dist_titulos_list = titulos_list.group_by('count').agg(pl.count().alias('frequency')).sort('count',descending=True)

In [24]:
'''
plt.bar(dist_titulos_list['count'].to_list(), dist_titulos_list['frequency'].to_list())
plt.yscale('log')
plt.show()
'''

"\nplt.bar(dist_titulos_list['count'].to_list(), dist_titulos_list['frequency'].to_list())\nplt.yscale('log')\nplt.show()\n"

---
## Analizar anomalias de la forma en la que se distribuyen los volumnes de agua de acuerdo a lo reportado en df_subterraneas y df_superficiales
---

In [25]:
df_subterraneas = data_loader.load_subterraneas()

In [26]:
df_subterraneas.filter(pl.col('titulo') == '08JAL106664/12IMGR98').head(4)

titulo,clave_de_region_hidrologica,nombre_de_region_hidrologica,clave_de_estado,nombre_de_estado,clave_de_municipio,nombre_de_municipio,clave_de_cuenca_hidrologica,nombre_de_cuenca_hidrologica,volumen_anual_en_m3,clave_de_acuifero_que_menciona_el_titulo,acuifero_que_menciona_el_titulo,numero_de_anexo,clave_de_acuifero_homologado,nombre_de_acuifero_homologado,latitud,longitud,ultimo_registro
str,f64,str,i64,str,i64,str,i64,str,f64,i64,str,i64,f64,str,f64,f64,str
"""08JAL106664/12IMGR98""",12.0,"""LERMA SANTIAGO""",14,"""JALISCO""",53,"""LAGOS DE MORENO""",11,"""RIO VERDE GRANDE""",19825.0,1412,"""9B 20 DE NOVIEMBRE""",1,1412.0,"""20 DE NOVIEMBRE""",21.502778,-102.916389,"""abril 2019"""
"""08JAL106664/12IMGR98""",12.0,"""LERMA SANTIAGO""",14,"""JALISCO""",53,"""LAGOS DE MORENO""",11,"""RIO VERDE GRANDE""",19825.0,1412,"""9B 20 DE NOVIEMBRE""",2,1412.0,"""20 DE NOVIEMBRE""",21.513333,-102.914167,"""abril 2019"""
"""08JAL106664/12IMGR98""",12.0,"""LERMA SANTIAGO""",14,"""JALISCO""",53,"""LAGOS DE MORENO""",11,"""RIO VERDE GRANDE""",19825.0,1412,"""9B 20 DE NOVIEMBRE""",1,1412.0,"""20 DE NOVIEMBRE""",21.502778,-102.916389,"""octubre 2019"""
"""08JAL106664/12IMGR98""",12.0,"""LERMA SANTIAGO""",14,"""JALISCO""",53,"""LAGOS DE MORENO""",11,"""RIO VERDE GRANDE""",19825.0,1412,"""9B 20 DE NOVIEMBRE""",2,1412.0,"""20 DE NOVIEMBRE""",21.513333,-102.914167,"""octubre 2019"""


In [27]:
df_concesiones.filter(pl.col('titulo') == '08JAL106664/12IMGR98').head(2)

titulo,titular,ultimo_registro,uso_que_ampara_el_titulo,volumen_de_extraccion_anual_de_aguas_nacionales_que_ampara_el_titulo_en_m3,aprovechamientos_superficiales_que_ampara_el_titulo,volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3,aprovechamientos_subterraneos_que_ampara_el_titulo,volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3,puntos_de_descarga_que_ampara_el_titulo,volumen_de_descarga_diario_en_m3,zonas_federales_que_ampara_el_titulo,superficie_en_m2,fecha_de_registro,fecha_de_inicio_de_vigencia,fecha_de_termino_de_vigencia,autoridad_que_emite_el_acto,anio,mes
str,str,date,str,f64,i64,f64,i64,f64,i64,f64,i64,f64,str,str,str,str,i16,str
"""08JAL106664/12IMGR98""","""METODIO GALLO RODRIGUEZ""",2019-04-01,"""DIFERENTES USOS""",39650.0,0,0.0,2,39650.0,0,0.0,0,0.0,"""1998-01-27""",null,null,null,2019,"""4"""
"""08JAL106664/12IMGR98""","""METODIO GALLO RODRIGUEZ""",2019-10-01,"""DIFERENTES USOS""",39650.0,0,0.0,2,39650.0,0,0.0,0,0.0,"""1998-01-27""",null,null,null,2019,"""10"""


In [28]:
df_superficiales = data_loader.load_superficiales()

### Primero comparamos el numero de titulos de cada dataset y vemos que disntan entre si

In [29]:
print("\nTÍTULOS ÚNICOS EN CADA DATAFRAME")
titulos_concesiones = set(df_concesiones.select('titulo').unique().to_series().to_list())
titulos_subterraneas = set(df_subterraneas.select('titulo').unique().to_series().to_list())
titulos_superficiales = set(df_superficiales.select('titulo').unique().to_series().to_list())

print(f"Títulos únicos en df_concesiones: {len(titulos_concesiones)}")
print(f"Títulos únicos en df_subterraneas: {len(titulos_subterraneas)}")
print(f"Títulos únicos en df_superficiales: {len(titulos_superficiales)}")

# Intersecciones
titulos_en_subterraneas_y_concesiones = titulos_concesiones.intersection(titulos_subterraneas)
titulos_en_superficiales_y_concesiones = titulos_concesiones.intersection(titulos_superficiales)
titulos_en_todos = titulos_concesiones.intersection(titulos_subterraneas).intersection(titulos_superficiales)

print(f"\nTítulos que aparecen en concesiones Y subterráneas: {len(titulos_en_subterraneas_y_concesiones)}")
print(f"Títulos que aparecen en concesiones Y superficiales: {len(titulos_en_superficiales_y_concesiones)}")
print(f"Títulos que aparecen en los TRES dataframes: {len(titulos_en_todos)}")

print(f'\n Es decir hay {len(titulos_en_todos)} títulos que tienen ambos tipos de suministro, superficial y subterráneo.')
print(f' Concesiones que solo tienen un suministro subterráneo: {len(titulos_en_subterraneas_y_concesiones)}')
print(f' Concesiones que solo tienen un suministro superficial: {len(titulos_en_superficiales_y_concesiones)}')



TÍTULOS ÚNICOS EN CADA DATAFRAME
Títulos únicos en df_concesiones: 592469
Títulos únicos en df_subterraneas: 356989
Títulos únicos en df_superficiales: 130314

Títulos que aparecen en concesiones Y subterráneas: 356988
Títulos que aparecen en concesiones Y superficiales: 130314
Títulos que aparecen en los TRES dataframes: 909

 Es decir hay 909 títulos que tienen ambos tipos de suministro, superficial y subterráneo.
 Concesiones que solo tienen un suministro subterráneo: 356988
 Concesiones que solo tienen un suministro superficial: 130314


In [30]:
# Procesar fechas en los dataframes subterráneas y superficiales (similar a concesiones)
df_subterraneas = manipulator.date_string_to_date(df_subterraneas)
df_superficiales = manipulator.date_string_to_date(df_superficiales)

# Truncar fechas para consistencia
df_subterraneas = df_subterraneas.with_columns(
    pl.col('ultimo_registro').dt.date().alias('ultimo_registro')
)
df_superficiales = df_superficiales.with_columns(
    pl.col('ultimo_registro').dt.date().alias('ultimo_registro')
)

In [31]:
# ANÁLISIS DE COMPARACIÓN DE APROVECHAMIENTOS Y VOLÚMENES POR TÍTULO

def comparar_aprovechamientos_y_volumenes(titulos_concesiones,titulos_subterraneas,titulos_superficiales):
    """
    Compara los aprovechamientos y volúmenes entre df_concesiones, df_subterraneas y df_superficiales
    por título a lo largo de los períodos.
    """
    
    # Identificar columnas de volumen en cada dataframe
    vol_cols_concesiones = [col for col in df_concesiones.columns if 'volumen' in col.lower()]
    vol_cols_subterraneas = [col for col in df_subterraneas.columns if 'volumen' in col.lower()]
    vol_cols_superficiales = [col for col in df_superficiales.columns if 'volumen' in col.lower()]

    print("COLUMNAS RELACIONADAS CON EL VOLUMEN")
    print("df_concesiones:")
    for col in vol_cols_concesiones:
        print(f"  - {col}")
    print("\ndf_subterraneas:")
    for col in vol_cols_subterraneas:
        print(f"  - {col}")
    print("\ndf_superficiales:")
    for col in vol_cols_superficiales:
        print(f"  - {col}")
    
    
    # Títulos para comparables
    titulos_sub_vs_conc = titulos_concesiones.intersection(titulos_subterraneas)
    titulos_sup_vs_conc = titulos_concesiones.intersection(titulos_superficiales)
    
    print(f"\nCantidad de titulos compalabres")
    print(f"Títulos en concesiones Y subterráneas: {len(titulos_sub_vs_conc)}")
    print(f"Títulos en concesiones Y superficiales: {len(titulos_sup_vs_conc)}")
    
    return titulos_sub_vs_conc, titulos_sup_vs_conc, vol_cols_concesiones, vol_cols_subterraneas, vol_cols_superficiales

# Ejecutar análisis inicial


In [32]:
titulos_sub_vs_conc, titulos_sup_vs_conc, vol_cols_concesiones, vol_cols_subterraneas, vol_cols_superficiales = comparar_aprovechamientos_y_volumenes(titulos_concesiones,titulos_subterraneas,titulos_superficiales)

COLUMNAS RELACIONADAS CON EL VOLUMEN
df_concesiones:
  - volumen_de_extraccion_anual_de_aguas_nacionales_que_ampara_el_titulo_en_m3
  - volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3
  - volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3
  - volumen_de_descarga_diario_en_m3

df_subterraneas:
  - volumen_anual_en_m3

df_superficiales:
  - volumen_anual_en_m3

Cantidad de titulos compalabres
Títulos en concesiones Y subterráneas: 356988
Títulos en concesiones Y superficiales: 130314


In [33]:
# ANÁLISIS DETALLADO DE INCONSISTENCIAS

def analizar_inconsistencias_subterraneas():
    """
    Compara df_concesiones vs df_subterraneas para el mismo título y período
    """
    print("ANÁLISIS DE INCONSISTENCIAS: CONCESIONES vs SUBTERRÁNEAS (MISMO  TITULO Y PERIODO)")
    
    # Preparar datos para comparación
    concesiones_sub = df_concesiones.filter(
        pl.col('titulo').is_in(list(titulos_sub_vs_conc)) #Filtramos para los titulos comparables
    ).select([
        'titulo', 
        'ultimo_registro', 
        'volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3'
    ]).rename({
        'volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3': 'vol_subterraneo_concesiones_anual'
    })
    
    subterraneas_comp = df_subterraneas.select([
        'titulo', 
        'ultimo_registro', 
        'volumen_anual_en_m3'
    ]).group_by(['titulo','ultimo_registro']).agg([
        pl.col('volumen_anual_en_m3').cast(pl.Float64, strict=False).sum().alias('vol_subterraneo_directo_anual')
    ])
    
    # Join para comparar
    comparacion_vol_subterranea = concesiones_sub.join(
        subterraneas_comp, 
        on=['titulo', 'ultimo_registro'],  #join solo en estos campos  que coincidan 
        how='inner'
    ).with_columns([
        # Convertir a numérico y manejar nulos
        pl.when(pl.col('vol_subterraneo_concesiones_anual').is_null())
        .then(0.0)
        .otherwise(pl.col('vol_subterraneo_concesiones_anual').cast(pl.Float64, strict=False))
        .alias('vol_conc_num'), #Se crea esta columna para el volumen subterráneo de _concesiones

        pl.when(pl.col('vol_subterraneo_directo_anual').is_null())
        .then(0.0)
        .otherwise(pl.col('vol_subterraneo_directo_anual').cast(pl.Float64, strict=False))
        .alias('vol_sub_num') #Se crea esta columna para el volumen subterráneo de reportados
    ]).with_columns([
        # Calcular diferencias 
        (pl.col('vol_conc_num') - pl.col('vol_sub_num')).alias('diferencia_volumen'),
        pl.when(pl.col('vol_conc_num') == pl.col('vol_sub_num'))
        .then(True)
        .otherwise(False)
        .alias('volumenes_coinciden')
    ])
    #La función height de polars te da el número de filas 
    total_registros = comparacion_vol_subterranea.height
    registros_coinciden = comparacion_vol_subterranea.filter(pl.col('volumenes_coinciden')).height
    registros_difieren = total_registros - registros_coinciden
    
    print(f"Total de registros comparables: {total_registros:,}")
    print(f"Registros con volúmenes coincidentes: {registros_coinciden:,} ({registros_coinciden/total_registros*100:.2f}%)")
    print(f"Registros con volúmenes diferentes: {registros_difieren:,} ({registros_difieren/total_registros*100:.2f}%)")
    
    return comparacion_vol_subterranea

In [34]:
def analizar_inconsistencias_superficiales():
    """
    Compara df_concesiones vs df_superficiales para el mismo título y período
    """
    print("\nANÁLISIS DE INCONSISTENCIAS: CONCESIONES vs SUPERFICIALES(MISMO  TITULO Y PERIODO)")
    
    # Preparar datos para comparación
    concesiones_sup = df_concesiones.filter(
        pl.col('titulo').is_in(list(titulos_sup_vs_conc))
    ).select([
        'titulo', 
        'ultimo_registro', 
        'volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3'
    ]).rename({
        'volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3': 'vol_superficial_concesiones'
    })
    
    superficiales_comp = df_superficiales.select([
        'titulo', 
        'ultimo_registro', 
        'volumen_anual_en_m3'
    ]).group_by(['titulo','ultimo_registro']).agg([
        pl.col('volumen_anual_en_m3').cast(pl.Float64, strict=False).sum().alias('vol_superficial_directo')
    ])

    # Join para comparar
    comparacion_vol_superficial = concesiones_sup.join(
        superficiales_comp, 
        on=['titulo', 'ultimo_registro'], 
        how='inner'
    ).with_columns([
        # Convertir a numérico y manejar nulos
        pl.when(pl.col('vol_superficial_concesiones').is_null())
        .then(0.0)
        .otherwise(pl.col('vol_superficial_concesiones').cast(pl.Float64, strict=False))
        .alias('vol_conc_num'),
        
        pl.when(pl.col('vol_superficial_directo').is_null())
        .then(0.0)
        .otherwise(pl.col('vol_superficial_directo').cast(pl.Float64, strict=False))
        .alias('vol_sup_num')
    ]).with_columns([
        # Calcular diferencias
        (pl.col('vol_conc_num') - pl.col('vol_sup_num')).alias('diferencia_volumen'),
        pl.when(pl.col('vol_conc_num') == pl.col('vol_sup_num'))
        .then(True)
        .otherwise(False)
        .alias('volumenes_coinciden')
    ])
    
    total_registros = comparacion_vol_superficial.height
    registros_coinciden = comparacion_vol_superficial.filter(pl.col('volumenes_coinciden')).height
    registros_difieren = total_registros - registros_coinciden
    
    print(f"Total de registros comparables: {total_registros:,}")
    print(f"Registros con volúmenes coincidentes: {registros_coinciden:,} ({registros_coinciden/total_registros*100:.2f}%)")
    print(f"Registros con volúmenes diferentes: {registros_difieren:,} ({registros_difieren/total_registros*100:.2f}%)")
    
    return comparacion_vol_superficial



In [35]:
# Ejecutar análisis
comp_subterraneas = analizar_inconsistencias_subterraneas()


ANÁLISIS DE INCONSISTENCIAS: CONCESIONES vs SUBTERRÁNEAS (MISMO  TITULO Y PERIODO)
Total de registros comparables: 5,994,892
Registros con volúmenes coincidentes: 5,969,240 (99.57%)
Registros con volúmenes diferentes: 25,652 (0.43%)


In [36]:
comp_superficiales = analizar_inconsistencias_superficiales()


ANÁLISIS DE INCONSISTENCIAS: CONCESIONES vs SUPERFICIALES(MISMO  TITULO Y PERIODO)
Total de registros comparables: 2,484,219
Registros con volúmenes coincidentes: 2,478,184 (99.76%)
Registros con volúmenes diferentes: 6,035 (0.24%)


In [ ]:
# Crear métricas de discrepancias binarias para subterráneas
discrepancias_sub_binarias = comp_subterraneas.group_by('titulo').agg([
    # Si hay al menos una discrepancia, marcar como 1, sino 0
    pl.when(pl.col('volumenes_coinciden').sum() < pl.col('volumenes_coinciden').count())
    .then(1) #1 si hay un discrepancia
    .otherwise(0) #0 si no hay discrepancia
    .alias('Discrepancia_V_Subterraneo')
])

# Crear métricas de discrepancias binarias para superficiales
discrepancias_sup_binarias = comp_superficiales.group_by('titulo').agg([
    # Si hay al menos una discrepancia, marcar como 1, sino 0
    pl.when(pl.col('volumenes_coinciden').sum() < pl.col('volumenes_coinciden').count())
    .then(1) #1 si hay un discrepancia
    .otherwise(0) #0 si no hay discrepancia
    .alias('Discrepancia_V_Superficial')
])
# Si aparece NULL es que en esos caso el titulo correspondiente no recurre de dicho  tipo de  suministro 





In [39]:
# Agregar las columnas al df_anomalias
df_anomalias = df_anomalias.join(
    discrepancias_sub_binarias, 
    on='titulo', 
    how='left'
).join(
    discrepancias_sup_binarias, 
    on='titulo', 
    how='left'
)

---
## Análisis de anomlias en el df_concesiones contrastando la suma de suministros con le  total
---

In [40]:
df_concesiones.columns

['titulo',
 'titular',
 'ultimo_registro',
 'uso_que_ampara_el_titulo',
 'volumen_de_extraccion_anual_de_aguas_nacionales_que_ampara_el_titulo_en_m3',
 'aprovechamientos_superficiales_que_ampara_el_titulo',
 'volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3',
 'aprovechamientos_subterraneos_que_ampara_el_titulo',
 'volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3',
 'puntos_de_descarga_que_ampara_el_titulo',
 'volumen_de_descarga_diario_en_m3',
 'zonas_federales_que_ampara_el_titulo',
 'superficie_en_m2',
 'fecha_de_registro',
 'fecha_de_inicio_de_vigencia',
 'fecha_de_termino_de_vigencia',
 'autoridad_que_emite_el_acto',
 'anio',
 'mes']

In [42]:
inconsistencias_suma_volumenes = df_concesiones.with_columns([
    # Convertir a numérico y manejar nulos
    pl.when(pl.col('volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3').is_null())
    .then(0.0)
    .otherwise(pl.col('volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3').cast(pl.Float64, strict=False))
    .alias('vol_superficial_num'),
    
    pl.when(pl.col('volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3').is_null())
    .then(0.0)
    .otherwise(pl.col('volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3').cast(pl.Float64, strict=False))
    .alias('vol_subterraneo_num'),
    
    pl.when(pl.col('volumen_de_extraccion_anual_de_aguas_nacionales_que_ampara_el_titulo_en_m3').is_null())
    .then(0.0)
    .otherwise(pl.col('volumen_de_extraccion_anual_de_aguas_nacionales_que_ampara_el_titulo_en_m3').cast(pl.Float64, strict=False))
    .alias('vol_total_num')
]).with_columns([
    # Calcular la suma de superficial + subterráneo
    (pl.col('vol_superficial_num') + pl.col('vol_subterraneo_num')).alias('suma_volumenes'),
    
    # Verificar si la suma coincide con el total
    pl.when((pl.col('vol_superficial_num') + pl.col('vol_subterraneo_num')) == pl.col('vol_total_num'))
    .then(True)
    .otherwise(False)
    .alias('suma_coincide_con_total')
]).select(['titulo', 'ultimo_registro', 'vol_superficial_num', 'vol_subterraneo_num', 'vol_total_num', 'suma_volumenes', 'suma_coincide_con_total'])

# Crear métrica binaria por título: 1 si hay al menos una inconsistencia, 0 si todas coinciden
discrepancias_suma_binarias = inconsistencias_suma_volumenes.group_by('titulo').agg([
    pl.when(pl.col('suma_coincide_con_total').sum() < pl.col('suma_coincide_con_total').count())
    .then(1)  # 1 si hay al menos una discrepancia
    .otherwise(0)  # 0 si todas las sumas coinciden
    .alias('Discrepancia_Suma_Volumenes')
])

# Agregar la nueva columna al df_anomalias
df_anomalias = df_anomalias.join(
    discrepancias_suma_binarias,
    on='titulo',
    how='left'
)